In [58]:
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from sentence_transformers import SentenceTransformer
import glob
import os
from tqdm import tqdm
from typing import Generator, Tuple, Iterator

import faiss
import numpy as np
import pickle

In [59]:
def pickle_read(filename: str):
    with open("data.pkl", "rb") as f:
        loaded_data = pickle.load(f)
    return loaded_data
def pickle_write(data, filename: str):
    with open(filename + ".pkl", "wb") as f:
        pickle.dump(data, f)

In [4]:
def index_documents(knowledge_base_path: str = "raw-kb") -> Iterator[Tuple[str, str, int]]:
    for filepath in glob.glob(os.path.join(knowledge_base_path, "*.pdf")):
        filename = os.path.basename(filepath)
        with pdfplumber.open(filepath) as pdf:
            for idx, page in enumerate(pdf.pages):
                yield page.extract_text(), filename, idx

In [51]:
fine_splitter = SentenceTransformersTokenTextSplitter(
    model_name="all-mpnet-base-v2", #"sentence-transformers/all-MiniLM-L6-v2",
    tokens_per_chunk=1024,
    chunk_overlap=50
)

model = fine_splitter._model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [69]:
try:
    #1/0
    index = faiss.read_index("basic_rag.faiss")
    meta = pickle_read("meta")
    print("Index readed")
except:
    index = faiss.IndexFlatL2(fine_splitter._model[1].word_embedding_dimension)
    meta = {}
    print("Index created")

Index created


In [78]:
%%time
document_index = index.ntotal
meta = {}
for text, filename, page_index in tqdm(index_documents()):
    for chunk in fine_splitter.split_text(text):
        embeddings = model.encode([text])
        meta[document_index] = {"filename": filename, "page_index": page_index, "text": chunk}
        index.add(embeddings)
        document_index += 1
faiss.write_index(index, "basic_rag.faiss")
pickle_write(meta, "meta")

20it [00:07,  3.47it/s]Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 

CPU times: user 10min 29s, sys: 3min 36s, total: 14min 5s
Wall time: 2min 7s


In [72]:
scores, indexies = index.search(model.encode(["Philipp Schindler"]), k=10)

In [73]:
for score, idx in list(zip(scores[0], indexies[0])):
    print(score, meta[idx])

KeyError: -1

In [77]:
index.ntotal

0